## 단어자동완성
이번에는 RNN을 이용해 영단어 4글자중 3글자만 주면 나머지 한글자를 추천하는  프로그램을 만들어 봅시다.   

학습시킬 데이터는 영문자로 구성된 임의의 단어를 사용하며 한 글자당 하나의 단계로 볼 수 있습니다. 즉, 한 글자가 한 단계의 입력값이 되며 총 글자수가 총 단계수가 됩니다.   

참고로 dynamic_rnn의 sequence_length 옵션을 사용하면 가변 길이 단어를 학습시킬수 있습니다. 짧은단어는 가장 긴 단어의 길이 만큼 뒷부분을 0으로 채우고 해당 단어의 길이를 계산해 sequence_length로 넘겨주면 됩니다(batch_size만큼의 배열로)

In [0]:
import tensorflow as tf
import numpy as np

char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
num_dic = {n: i for i , n in enumerate(char_arr)} #{'a':0, 'b':1, ....}
dic_len = len(num_dic)

입력으로 알파벳 순서에서 각 글자에 해당하는 인덱스를 원핫 인코딩으로 표현한 값을 취할겁니다.

In [0]:
seq_data = ['word','wood','deep','dive','cold','cool','load','love','kiss','kind']

학습에 사용할 단어를 배열로 저장합니다.

In [0]:
def make_batch(seq_data):
  input_batch =[]
  target_batch = []

  for seq in seq_data:
    input = [num_dic[n] for n in seq[:-1]]
    target = num_dic[seq[-1]]
    input_batch.append(np.eye(dic_len)[input])
    target_batch.append(target)

  return input_batch, target_batch

 ```python
 input = [num_dic[n] for n in seq[:-1]]
```
입력값 용으로 단어의 처음 세글자의 알파벳 인덱스를 구한 배열을 만듭니다. ex) deep -> d,e,e -> [3,4,4]

```python
 target = num_dic[seq[-1]]
 ```
 출력값 용으로 마지막 글자의 알파벳 인덱스를 구합니다. ex) deep -> p -> [15]   
 출력값을 원핫 인코딩하지 않는 이유는 후에 나옵니다.

 ```python
 input_batch.append(np.eye(dic_len)[input])
 ```
 입력값을 원핫 인코딩으로 변경하여 넣습니다.


In [0]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

신경망 모델 구성에 앞서 먼저 옵션들을 설정해줍니다.   
단어의 앞 3글자를 학습 시킬것이므로 **n_step**=3이 됩니다.   
입력값과 출력값은 원-핫 인코딩을 사용할 것이므로 알파벳 전체 배열크기인 **dic_len**과 같습니다.   

In [0]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

Y의 placeholder는 원-핫 인코딩이 아닌 인덱스 숫자를 그대로 사용하기에 batch_size에 해당하는 하나의 차원만 있는것에 유의 합니다. 

In [0]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

다음으로 여러셀을 조합해 심층 신경망을 만들기 위해 2개의 RNN 셀을 생성합니다.   
추가로 **``DropoutWrapper``**을 이용해 과적합 방지를 위한 드롭아웃 기법을 쉽게 적용할수 있습니다.   

In [0]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

그런 다음 앞서 만든 셀들을 **``8MultiRNNCell88``**함수를 사용하여 조합하고 **``8dynamic_rnn``** 함수를 사용하여 심층 순환 신경망(Deep RNN)을 만듭니다.

In [0]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W)+b

In [0]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits= model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

손실함수를 만들 때 **``sparse_softmax_cross_entropy_with_logits``**를 사용하여 labels=Y에서 Y가 인덱스 숫자값인 것을 자동으로 원-핫 인코딩값으로 변환하여 계산해줍니다. 

이제 신경망을 학습시켜 봅시다.   
make_batch 함수를 이용하여 seq_data에 저장한 단어들을 입력값(첫 3글자)과 실측값(마지막 한글자)으로 분리하고, 이 값들을 최적화 함수를 실행하는 코드에 넣어 신경망을 학습시킵니다.

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
  _, loss = sess.run([optimizer,cost],feed_dict = {X: input_batch, Y: target_batch})

  print('Epoch:', '%04d'%(epoch),'cost=', '{:.6f}'.format(loss))

print('최적화 완료')

Epoch: 0001 cost= 3.977742
Epoch: 0002 cost= 2.989139
Epoch: 0003 cost= 1.610842
Epoch: 0004 cost= 1.711844
Epoch: 0005 cost= 0.983833
Epoch: 0006 cost= 1.092157
Epoch: 0007 cost= 0.731147
Epoch: 0008 cost= 0.556994
Epoch: 0009 cost= 0.396556
Epoch: 0010 cost= 0.465858
Epoch: 0011 cost= 0.356636
Epoch: 0012 cost= 0.366594
Epoch: 0013 cost= 0.255504
Epoch: 0014 cost= 0.300342
Epoch: 0015 cost= 0.394962
Epoch: 0016 cost= 0.289838
Epoch: 0017 cost= 0.154152
Epoch: 0018 cost= 0.235903
Epoch: 0019 cost= 0.116413
Epoch: 0020 cost= 0.126386
Epoch: 0021 cost= 0.113128
Epoch: 0022 cost= 0.103428
Epoch: 0023 cost= 0.100863
Epoch: 0024 cost= 0.070067
Epoch: 0025 cost= 0.048491
Epoch: 0026 cost= 0.038023
Epoch: 0027 cost= 0.038658
Epoch: 0028 cost= 0.022490
Epoch: 0029 cost= 0.104551
Epoch: 0030 cost= 0.046892
최적화 완료


이번에는 결과값으로 예측한 단어를 정확도와 함께 출력해 봅시다.   
Y가 정수 값이므로 prediction역시 argmax로 나온값을 int(정수)로 바꾸어 prediction_check에 사용합니다.   

그런다음 make_batch함수를 이용해 학습에 사용할 단어를 받고 예측모델을 돌립니다.   

마지막으로 모델이 예측한 값을 가지고 각각의 값에 해당하는 인덱스의 알파벳을 가져와 예측한 단어를 출력합니다.

In [14]:
prediction = tf.cast(tf.argmax(model,1),tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
  last_char = char_arr[predict[idx]]
  predict_words.append(val[:3]+last_char)

print('\n=== 예측결과 ===')
print('입력값', [w[:3]+' ' for w in seq_data])
print('예측값',predict_words)
print('정확도', accuracy_val)


=== 예측결과 ===
입력값 ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값 ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도 1.0
